# SkyPortal Access to ZTF Alerts

In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import datetime
from datetime import date

from astropy.time import Time

In [2]:
mjd = 59319
mag_cut = 20.5

Time(mjd, format='mjd').iso

'2021-04-15 00:00:00.000'

## Set up the SkyPortal Query

All-sky search for ZTF alerts from Ampel:
- Look for all alerts within the past 24 hours.
- Apply the query using the SkyPortal instance at LBNL.

In [3]:
today = date.today()
DD = datetime.timedelta(days=1)
startDate = today - DD
token = '7fa6f13d-9055-4cbe-9514-ffc87e63b7dd'

print(startDate)

2021-03-31


In [4]:
query = 'http://desi2.lbl.gov:5000/api/sources?startDate=%s&origin=Ampel' % (startDate)
print(query)

response = requests.get(
#     'http://desi2.lbl.gov:5000/api/sources?ra=10&dec=00&radius=5&startDate=%s' % (startDate),
    query,
    headers={'Authorization': f'token {token}'}
)

print(f'HTTP code: {response.status_code}, {response.reason}')

http://desi2.lbl.gov:5000/api/sources?startDate=2021-03-31&origin=Ampel
HTTP code: 200, OK


In [5]:
data = response.json()['data']
matches = data['totalMatches']
sources = data['sources']
# alerts = pd.read_json(sources)
# ids = [sub['id'] for sub in sources]
# print(ids)

In [6]:
matches

75

## Create ToO Ledger

In [7]:
epoch = 2000.0
for da in sources:
    query = 'http://3.238.105.175:8081/parametric/sn?oid=%s&mjd=%s' % (da['id'], mjd)
    response = requests.get(query)
    forecast = response.json()['forecast']
    magpsf = forecast[1]['magpsf']
    
    if magpsf is not None:
        if magpsf <= mag_cut:
            prog = 'BRIGHT' if forecast[1]['magpsf'] < 21 else 'DARK'
            print('{:<10.6f} {:>10.6f} {:>9.6f} {:>9.6f} {:>7.1f}  AGK  FIBER  {:7s} {:>15.8f} {:>15.8f}'.format(
                da['ra'], da['dec'], 0, 0, epoch, prog, mjd, mjd+14))

237.258828  36.581008  0.000000  0.000000  2000.0  AGK  FIBER  BRIGHT   59319.00000000  59333.00000000
286.956527  62.963744  0.000000  0.000000  2000.0  AGK  FIBER  BRIGHT   59319.00000000  59333.00000000
